### Days: wps_climdex_get_available_indices

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf)'s [climdex.get.available.indices](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=9) function

This function takes a climdexInput object as input and returns the names of all the indices which may be computed or, ifget.function.namesis TRUE (the default), the names of the functions corresponding to the indices.

In [1]:
import json
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from importlib.resources import files
from tempfile import NamedTemporaryFile

from wps_tools.testing import get_target_url
from wps_tools.R import test_rda_output

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_get_available_indices?

Signature:
quail.climdex_get_available_indices(
    climdex_input=None,
    ci_name='ci',
    loglevel='INFO',
    output_file='output.rda',
    output_formats=None,
)
Docstring:
Returns the names of all the indices which may be computed or, if get_function_names is TRUE, the names of the functions corresponding to the indices.

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    RDS or Rdata (.rds, .rda, .rdata) file containing R Object of type climdexInput
ci_name : string
    Name of the climdexInput object. Only needed when using Rdata input. For RDS input it may be left as the default value.
output_file : string
    Filename to store the output Rdata (extension .rda)
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
avail_processes : string
    Available climdex indices (values) and the processes to use to compute them (keys)
File:      ~/github/quail/</tmp/quail-venv/lib/python3.8/site-p

#### Run wps_climdex_get_available_indices Process with rda input

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="indices_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_get_available_indices(
            climdex_input=(files("tests") / "data/climdexInput.rda").resolve(),
            ci_name="ci",
            output_file=output_file.name,
        )
avail_processes = eval(output.get()[0])

In [7]:
# NBVAL_IGNORE_OUTPUT
avail_processes

{'wps_climdex_days': ['su', 'id', 'fd', 'tr'],
 'wps_climdex_gsl': ['gsl'],
 'wps_climdex_mmdmt': ['txx', 'tnx', 'txn', 'tnn'],
 'wps_climdex_rmm': ['r10mm', 'r20mm', 'rnnmm'],
 'wps_climdex_spells': ['cdd', 'csdi', 'cwd', 'wsdi'],
 'wps_climdex_temp_pctl': ['tn10p', 'tn90p', 'tx10p', 'tx90p'],
 'wps_climdex_dtr': ['dtr'],
 'wps_climdex_ptot': ['r95ptot', 'r99ptot', 'prcptot'],
 'wps_climdex_sdii': ['sdii'],
 'wps_climdex_rxnday': ['rx1day', 'rx5day']}

### You can also pass an rds file

In [8]:
with NamedTemporaryFile(suffix=".rda", prefix="indices_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_get_available_indices(
            climdex_input=(files("tests") / "data/climdexInput.rds").resolve(),
            ci_name="ci",
            output_file=output_file.name,
        )

#### Test output against expected output

In [9]:
with open("tests/data/expected_avail_processes.json","r") as f:
    expected_avail_processes = json.load(f)
    
for key, value in expected_avail_processes.items():
    assert avail_processes[key] == value